In [ ]:
import torch
import torch.nn.functional as F
from PIL import Image

from input_pipeline import Images
from generator import Generator

In [ ]:
DEVICE = torch.device('cuda:1')

SIZE = 1024
WEIGHTS = 'models/run01_epoch_1500_generator.pth'
DATA = '/home/dan/datasets/DIV2K/DIV2K_valid_HR/'

In [ ]:
def downsample(images):
    return F.interpolate(
        images, mode='bilinear',
        size=(SIZE // 4, SIZE // 4),
        align_corners=False
    )


dataset = Images(DATA, SIZE, is_training=False)
G = Generator().eval().to(DEVICE)
state = torch.load(WEIGHTS, map_location=DEVICE)
G.load_state_dict(state)

# Load an image

In [ ]:
B = dataset[58].unsqueeze(0).to(DEVICE) # 64, 58
A = downsample(B)

image = B.squeeze(0).cpu().permute(1, 2, 0).numpy()
image = (image * 255).astype('uint8')

downsampled_image = A.squeeze(0).cpu().permute(1, 2, 0).numpy()
downsampled_image = (downsampled_image * 255).astype('uint8')

Image.fromarray(downsampled_image)

In [ ]:
Image.fromarray(image)

# Upscale

In [ ]:
with torch.no_grad():
    B_restored = G(A)

image = B_restored.squeeze(0).permute(1, 2, 0).cpu().numpy()
image = (image * 255).astype('uint8')
Image.fromarray(image)

In [ ]:
Image.fromarray(downsampled_image).resize((SIZE, SIZE), Image.LANCZOS)